# Assigment 7

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

### 1.Import data from PDF Format

In [ ]:
# To install Java and set the path on Windows, follow these steps:

# 1. Download the latest version of Java from: https://www.oracle.com/java/technologies/downloads
# 2. Install Java on your computer
# 3. Get the direction of java. Type in CMD: where java
# 4. Copy the last result, it will be <particular_path>
# 5. Type in CMD: setx PATH "%PATH%;<particular_path>"
# 6. Check if the installation was successful by typing "java -version" in CMD

In [28]:
#!pip install tabula
#!pip install --upgrade tabula-py
#!pip install PyPDF2
import io
import requests
#import PyPDF2
import pandas as pd
from functools import reduce
import tabula

In [85]:
link = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'
bbva_dir = read_pdf(link, pages="all", stream=True, encoding='ISO-8859-1')

In [86]:
for page in range(0,len(bbva_dir)):
    # Obtener el número de columnas 
    n_var = len(bbva_dir[page].columns)
    
    # especificar limpieza para la primera página
    if page == 0:
        # Usamos la primera fila como nombres de las columnas
        bbva_dir[page].columns = bbva_dir[page].iloc[0]
        # Almacenar los nombres de las columnas en un objeto
        column_names = bbva_dir[page].columns
        # Eliminar la primera fila, que contiene los nombres de las columnas
        bbva_dir[page] = bbva_dir[page].drop(bbva_dir[page].index[0])
        # Almacenar el dataframe preprocesado en un nuevo df 
        bbva_offices = bbva_dir[page]
        # Resetear el índice del nuevo dataframe 
        bbva_offices.reset_index()
    # especificar limpieza si el número de columnas es igual a 6 
    elif n_var == 6:
        # Eliminar columna innecesaria
        bbva_dir[page] = bbva_dir[page].drop('Unnamed: 0', axis=1)
        row = list(bbva_dir[page].columns)
        bbva_dir[page].loc[len(bbva_dir[page])] = row
        bbva_dir[page].columns = column_names
        last_row = bbva_dir[page].iloc[-1]
        bbva_dir[page] = bbva_dir[page].iloc[0:-1]
        bbva_dir[page] = pd.concat([last_row.to_frame().T, bbva_dir[page]], ignore_index=True)
        bbva_offices = pd.concat([bbva_dir[page], bbva_offices])
        bbva_offices.reset_index()
    # 
    else:
        row = list(bbva_dir[page].columns)
        bbva_dir[page].loc[len(bbva_dir[page])] = row
        bbva_dir[page].columns = column_names
        last_row = bbva_dir[page].iloc[-1]
        bbva_dir[page] = bbva_dir[page].iloc[0:-1]
        bbva_dir[page] = pd.concat([last_row.to_frame().T, bbva_dir[page]], ignore_index=True)
        # Concatenar la última fila con el dataframe preprocesado anteriormente
        bbva_offices = pd.concat([bbva_dir[page], bbva_offices])
        bbva_offices.reset_index()

In [88]:
# Filtrar las oficinas de LIMA
lima = bbva_offices['DEPARTAMENTO'] == "LIMA"
bbva_lima = bbva_offices[lima]
bbva_lima = bbva_lima.reset_index()
bbva_lima = bbva_lima.iloc[: , 1:]

# Crear columna con la dirección exacta
cols = [ 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'DIRECCION' ]
bbva_lima[ 'DIRECCION_EXACTA' ] = bbva_lima[ cols ].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
bbva_lima

,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO,DIRECCION_EXACTA
0,SANTA MONICA,AV. JUAN PEZET 1425,SAN ISIDRO,LIMA,LIMA,"LIMA, LIMA, SAN ISIDRO, AV. JUAN PEZET 1425"
1,SEBASTIÁN BARRANCA,"AV. IQUITOS 795-799, ESQ. CON SEBASTIÁN BARRAN...",LA VICTORIA,LIMA,LIMA,"LIMA, LIMA, LA VICTORIA, AV. IQUITOS 795-799, ..."
2,SEPARADORA INDUSTRIAL,AV. LA MOLINA 540,LA MOLINA,LIMA,LIMA,"LIMA, LIMA, LA MOLINA, AV. LA MOLINA 540"
3,STRIP CENTER PROCERES,AV. LOS PROCERES (ANTES AV. GUARDIA CIVIL) 100...,SANTIAGO DE SURCO,LIMA,LIMA,"LIMA, LIMA, SANTIAGO DE SURCO, AV. LOS PROCERE..."
4,SURQUILLO,AV. ANGAMOS ESTE 1256,SURQUILLO,LIMA,LIMA,"LIMA, LIMA, SURQUILLO, AV. ANGAMOS ESTE 1256"
...,...,...,...,...,...,...
196,AV. CENTRAL,AV. BASADRE 133,SAN ISIDRO,LIMA,LIMA,"LIMA, LIMA, SAN ISIDRO, AV. BASADRE 133"
197,AV. DEL EJERCITO,AV. DEL EJERCITO 789 ESQUINA CALLE JOSE TORIBI...,MIRAFLORES,LIMA,LIMA,"LIMA, LIMA, MIRAFLORES, AV. DEL EJERCITO 789 E..."
198,AV. PERU,AV. PERU 3442-3444,SAN MARTIN DE PORRES,LIMA,LIMA,"LIMA, LIMA, SAN MARTIN DE PORRES, AV. PERU 344..."
199,AV. SUCRE,AV. SUCRE 970,PUEBLO LIBRE,LIMA,LIMA,"LIMA, LIMA, PUEBLO LIBRE, AV. SUCRE 970"


In [53]:
#r = requests.get(link)
#f = io.BytesIO(r.content)
#
#reader = PyPDF2.PdfReader(f)
#page_content=[] 
#for i in range(len(reader.pages)):
#    page_content +=reader.pages[i].extract_text().strip().split('   ')
#
#res_ica = list(map(lambda st: str.replace(st, "ICA ", "ICA\t"), page_content))
#res_t = list(map(lambda st: str.replace(st, "  ", "\t"), res_ica))
#res2 = reduce(lambda x, y: x +"\t"+ y, res_t)
#res_per = res2. replace( "PASEO DE LA\tREPÚBLICA\tS/N - CCPLAZA LIMA SUR",
#                                            "PASEO DE LA REPÚBLICA S/N - CCPLAZA LIMA SUR") 
#
#res_ca = res_per.replace("CA.\tDERECHA\t370","CA. DERECHA 370")
#res_may = res_ca.replace("CALLE DOS\tDE MAYO Nº\t9 -PACASMAYO","CALLE DOS DE MAYO Nº 9 -PACASMAYO")
#res_3 = res_may.replace("URB. CACHIMAYO\tC -21-A\tSAN SEBASTIAN","URB. CACHIMAYO C -21-A SAN SEBASTIAN")
#res_4 = res_3.replace("CARRETERA\tPANAMERICANA\tURB. SANTO DOMINGO MZ C LOTE",
#                        "CARRETERA PANAMERICANA URB.SANTO DOMINGO MZ C LOTE")
#res_5 = res_4.replace("\tCANGALLO N. - 180 LA VICTORIA",
#                        "CANGALLO N. - 180\tLA VICTORIA")
#res_6 = res_5.replace("AV. PASEO LA REPÚBLICA\tS/N Y CRUCE CON AV. BOLIVIA S/N, LC \nLS-15 LIMA",
#                        "AV. PASEO LA REPÚBLICA S/N Y CRUCE CON AV. BOLIVIA S/N, LC \nLS-15\tLIMA")
#
#res_vf = res_6.replace("\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t", "")
#res_ofi = res_vf.replace("\tOFICINA", "\t\nOFICINA")
#base_to_df =res_ofi.strip().split('\t\n')
#res_f = list(map(lambda st: str.replace(st, "\n", ""), base_to_df))
#
#
#df = pd.DataFrame(res_f, columns = ['bbva_sedes'])
#df_ini = df['bbva_sedes'].str.split('\t', expand=True)
#df_sec = df['bbva_sedes'].str.split('\t', expand=True).dropna()
#lista_sec =list(df_sec.index.values)+[0]
#df_2 = df_ini.drop(lista_sec)
#df_2.rename(columns={0: 'OFICINA', 1: 'DIRECCION', 2 :'DISTRITO',3: 'PROVINCIA', 4: 'DEPARTAMENTO'}, inplace=True)
#df_3 = df_2.drop(columns=5)
#df_sec[6] = df_sec[1]+" "+df_sec[2]  
#df_sec_1 = df_sec.drop(columns=[1,2])
#df_sec_1.rename(columns={0: 'OFICINA', 6: 'DIRECCION', 3 :'DISTRITO',4: 'PROVINCIA', 5: 'DEPARTAMENTO'}, inplace=True)
#Df_oficinas = pd.concat([df_3,df_sec_1]).reset_index().drop(columns='index')
#bbva_lima = Df_oficinas[Df_oficinas["DEPARTAMENTO"]=="LIMA"].reset_index().drop(columns='index')

Overwriting cache for 0 3777


### 2. Use GoogleMaps API and geocode all the BBVA offices

In [89]:
# Importar las librerías necesarias y definir la clave del API
import googlemaps
gmaps = googlemaps.Client( key = 'insert_key' )

In [90]:
# Crear una muestra de la data total
bbva_lima_sample = bbva_lima.sample( frac = 0.05 ).copy()

In [91]:
# Definir la función para la geolocalización

def get_results( result_api ):

    try:
        lat = result_api[0]['geometry']['location']['lat']
        lon = result_api[0]['geometry']['location']['lng']   
    except:
        lat = np.nan
        lon = np.nan
    
    return ( lat, lon )

In [92]:
# Utilizar método lambda para aplicar la función "get_results"
# Crear una nueva columna denominada "COORDENADAS" que almacenará los valores de latitud y longitud
bbva_lima_sample[ 'COORDENADAS' ] = bbva_lima_sample.apply( lambda x: get_results( gmaps.geocode( x[ 'DIRECCION_EXACTA' ] , region = 'pe' ) ) , axis = 1 )

# Desdoblar la columna "COORDENADAS" en columnas separadas de latitud y longitud
bbva_lima_sample[ [ 'latitud' , 'longitud' ] ] = pd.DataFrame( bbva_lima_sample.COORDENADAS.tolist(), index = bbva_lima_sample.index )

bbva_lima_sample

,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO,DIRECCION_EXACTA,COORDENADAS,latitud,longitud
146,LA MARINA,AV. LA MARINA N° 2355 CENTRO COMERCIAL LA MARI...,SAN MIGUEL,LIMA,LIMA,"LIMA, LIMA, SAN MIGUEL, AV. LA MARINA N° 2355 ...","(-12.0782823, -77.0790397)",-12.078282,-77.079040
164,C.C.SUCRE,JR. LEGUIA Y MELENDEZ 998,PUEBLO LIBRE,LIMA,LIMA,"LIMA, LIMA, PUEBLO LIBRE, JR. LEGUIA Y MELENDE...","(-12.0754679, -77.064285)",-12.075468,-77.064285
81,LINCE,AV. IGNACIO MERINO 2099,LINCE,LIMA,LIMA,"LIMA, LIMA, LINCE, AV. IGNACIO MERINO 2099","(-12.0852949, -77.03225119999999)",-12.085295,-77.032251
40,PLAZA BUTTERS,PLAZA BUTTERS 301,BARRANCO,LIMA,LIMA,"LIMA, LIMA, BARRANCO, PLAZA BUTTERS 301","(-12.1436671, -77.01548609999999)",-12.143667,-77.015486
27,BIENESTAR DE LA MARINA,AV. VENEZUELA S/N CDRA 34,SAN MIGUEL,LIMA,LIMA,"LIMA, LIMA, SAN MIGUEL, AV. VENEZUELA S/N CDRA 34","(-12.0620255, -77.0924442)",-12.062026,-77.092444
35,PARINACOCHAS,"AV. MEXICO 1298, ESQ. PROLG. PARINACOCHAS",LA VICTORIA,LIMA,LIMA,"LIMA, LIMA, LA VICTORIA, AV. MEXICO 1298, ESQ....","(-12.0736141, -77.0164831)",-12.073614,-77.016483
157,C.C. FIORI,"AV. TOMÁS VALLE 499G, LC 14",SAN MARTIN DE PORRES,LIMA,LIMA,"LIMA, LIMA, SAN MARTIN DE PORRES, AV. TOMÁS VA...","(-12.0088873, -77.0602449)",-12.008887,-77.060245
143,LA COLMENA,AV. NICOLÁS DE PIÉROLA 649,LIMA,LIMA,LIMA,"LIMA, LIMA, LIMA, AV. NICOLÁS DE PIÉROLA 649","(-12.0496302, -77.0375281)",-12.049630,-77.037528
28,SEDAPAL,KM. 1 AUTOPISTA RAMIRO PRIALE (AL INTERIOR DE ...,EL AGUSTINO,LIMA,LIMA,"LIMA, LIMA, EL AGUSTINO, KM. 1 AUTOPISTA RAMIR...","(-12.0386756, -76.976219)",-12.038676,-76.976219
166,CAMACHO,AV. PROL. JAVIER PRADO ESTE 4921,LA MOLINA,LIMA,LIMA,"LIMA, LIMA, LA MOLINA, AV. PROL. JAVIER PRADO ...","(-12.0447042, -76.9343444)",-12.044704,-76.934344


In [93]:
# Identificamos las filas en las que las oficinas no pudieron ser geolocalizadas
bbva_lima_sample[ bbva_lima_sample [ 'latitud' ].isna() ]

,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO,DIRECCION_EXACTA,COORDENADAS,latitud,longitud


### 3. Use Google API to find the driving time

In [112]:
# Crear dataframe con información sobre cada integrante
dir_integrantes = [
    {'INTEGRANTE': 1, 'DIRECCION_EXACTA': 'Jose Balta 160, Comas'},
    {'INTEGRANTE': 2, 'DIRECCION_EXACTA': 'Av. Carlos Germán Amezaga 375, Cercado de Lima'},
    {'INTEGRANTE': 3, 'DIRECCION_EXACTA': 'Av. Universitaria 1801, San Miguel 15088'}
]

dir_integrantes = pd.DataFrame(dir_integrantes)

# Aplicar la función "get_results", guardar resultados en "COORDENADAS" y luego desdoblar la información
dir_integrantes[ 'COORDENADAS' ] = dir_integrantes.apply( lambda x: get_results( gmaps.geocode( x[ 'DIRECCION_EXACTA' ] , region = 'pe' ) ) , axis = 1 )
dir_integrantes[ [ 'latitud' , 'longitud' ] ] = pd.DataFrame( dir_integrantes.COORDENADAS.tolist(), index = dir_integrantes.index )
dir_integrantes

,INTEGRANTE,DIRECCION_EXACTA,COORDENADAS,latitud,longitud
0,1,"Jose Balta 160, Comas","(-11.9441574, -77.0475639)",-11.944157,-77.047564
1,2,"Av. Carlos Germán Amezaga 375, Cercado de Lima","(-12.1678904, -76.9745165)",-12.167890,-76.974516
2,3,"Av. Universitaria 1801, San Miguel 15088","(-12.0725123, -77.0812466)",-12.072512,-77.081247


In [113]:
for index, row in dir_integrantes.iterrows():
    lat1 = row['latitud']
    lng1 = row['longitud']
    origin = f'{lat1},{lng1}'
    print(origin)
    bbva_lima_sample[f'dir_{index}'] = pd.Series(dtype=float)
    
    for index2, row2 in bbva_lima_sample.iterrows():
        lat2 = row2['latitud']
        lng2 = row2['longitud']
        destination = f'{lat2},{lng2}'
        try:
            req2 = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&departure_time=now&traffic_model=best_guess&mode=driving&region=pe&key=insert_key")
            data_json = req2.json()
            bbva_lima_sample.at[index2, f'dir_{index}'] = data_json['routes'][0]['legs'][0]['duration']['value']
            time.sleep(2)
        except:
            print(f'dir_{index}_{index2}')
            continue

-11.9441574,-77.0475639
dir_0_146
dir_0_164
dir_0_81
dir_0_40
dir_0_27
dir_0_35
dir_0_157
dir_0_143
dir_0_28
dir_0_166
-12.1678904,-76.9745165
dir_1_146
dir_1_164
dir_1_81
dir_1_40
dir_1_27
dir_1_35
dir_1_157
dir_1_143
dir_1_28
dir_1_166
-12.0725123,-77.0812466
dir_2_146
dir_2_164
dir_2_81
dir_2_40
dir_2_27
dir_2_35
dir_2_157
dir_2_143
dir_2_28
dir_2_166


In [117]:
bbva_lima_sample = bbva_lima_sample.rename(columns={'dir_0': 'Integrante_1', 'dir_1': 'Integrante_2', 'dir_2': 'Integrante_3'})
bbva_lima_sample

,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO,DIRECCION_EXACTA,COORDENADAS,latitud,longitud,Integrante_1,Integrante_2,Integrante_3
146,LA MARINA,AV. LA MARINA N° 2355 CENTRO COMERCIAL LA MARI...,SAN MIGUEL,LIMA,LIMA,"LIMA, LIMA, SAN MIGUEL, AV. LA MARINA N° 2355 ...","(-12.0782823, -77.0790397)",-12.078282,-77.079040,3263.0,2439.0,510.0
164,C.C.SUCRE,JR. LEGUIA Y MELENDEZ 998,PUEBLO LIBRE,LIMA,LIMA,"LIMA, LIMA, PUEBLO LIBRE, JR. LEGUIA Y MELENDE...","(-12.0754679, -77.064285)",-12.075468,-77.064285,3306.0,2550.0,553.0
81,LINCE,AV. IGNACIO MERINO 2099,LINCE,LIMA,LIMA,"LIMA, LIMA, LINCE, AV. IGNACIO MERINO 2099","(-12.0852949, -77.03225119999999)",-12.085295,-77.032251,3385.0,1642.0,1561.0
40,PLAZA BUTTERS,PLAZA BUTTERS 301,BARRANCO,LIMA,LIMA,"LIMA, LIMA, BARRANCO, PLAZA BUTTERS 301","(-12.1436671, -77.01548609999999)",-12.143667,-77.015486,4474.0,1406.0,1870.0
27,BIENESTAR DE LA MARINA,AV. VENEZUELA S/N CDRA 34,SAN MIGUEL,LIMA,LIMA,"LIMA, LIMA, SAN MIGUEL, AV. VENEZUELA S/N CDRA 34","(-12.0620255, -77.0924442)",-12.062026,-77.092444,2907.0,2866.0,708.0
35,PARINACOCHAS,"AV. MEXICO 1298, ESQ. PROLG. PARINACOCHAS",LA VICTORIA,LIMA,LIMA,"LIMA, LIMA, LA VICTORIA, AV. MEXICO 1298, ESQ....","(-12.0736141, -77.0164831)",-12.073614,-77.016483,2758.0,1678.0,1804.0
157,C.C. FIORI,"AV. TOMÁS VALLE 499G, LC 14",SAN MARTIN DE PORRES,LIMA,LIMA,"LIMA, LIMA, SAN MARTIN DE PORRES, AV. TOMÁS VA...","(-12.0088873, -77.0602449)",-12.008887,-77.060245,1333.0,2356.0,1708.0
143,LA COLMENA,AV. NICOLÁS DE PIÉROLA 649,LIMA,LIMA,LIMA,"LIMA, LIMA, LIMA, AV. NICOLÁS DE PIÉROLA 649","(-12.0496302, -77.0375281)",-12.049630,-77.037528,2304.0,2135.0,1566.0
28,SEDAPAL,KM. 1 AUTOPISTA RAMIRO PRIALE (AL INTERIOR DE ...,EL AGUSTINO,LIMA,LIMA,"LIMA, LIMA, EL AGUSTINO, KM. 1 AUTOPISTA RAMIR...","(-12.0386756, -76.976219)",-12.038676,-76.976219,2698.0,1768.0,2939.0
166,CAMACHO,AV. PROL. JAVIER PRADO ESTE 4921,LA MOLINA,LIMA,LIMA,"LIMA, LIMA, LA MOLINA, AV. PROL. JAVIER PRADO ...","(-12.0447042, -76.9343444)",-12.044704,-76.934344,3134.0,1799.0,2943.0


### 4. Report closest and furthest offices